# PyTorch Leaf Disease Classification Pipeline

In [ ]:
# Cell 1: Install PyTorch
!pip install -q torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.9 MB/s eta 0:00:00


In [1]:
# Cell 2: Imports & Basic Setup
from pathlib import Path
import os, json, cv2, numpy as np, matplotlib.pyplot as plt
import torch
from PIL import Image
from shapely.geometry import Polygon

Real_DIR = Path(r"D:\rico\archive\annotated_apple_leaf_disease")
Mask_DIR = Path(r"D:\rico\archive\annotated_apple_leaf_disease")
IMG_SIZE = 224

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Create dummy image & mask :)
import numpy as np, cv2
from PIL import Image

# Dimensions
H, W = 224, 224

# 1) Dummy leaf image (random noise)
dummy_img = (np.random.rand(H, W, 3) * 255).astype(np.uint8)
# 2) Dummy mask: circle in center
dummy_mask = np.zeros((H, W), dtype=np.uint8)
cv2.circle(dummy_mask, (W//2, H//2), 50, 1, -1)

# Save to working directory
sample_img_path  = mask_DIR / "dummy_leaf.png"
sample_mask_path = mask_DIR / "dummy_leaf_mask.png"

Image.fromarray(dummy_img).save(sample_img_path)
Image.fromarray(dummy_mask * 255).save(sample_mask_path)

print(f"Dummy image → {sample_img_path}")
print(f"Dummy mask  → {sample_mask_path}")


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\rico\\archive\\annotated_apple_leaf_disease/dummy_leaf.png'

In [5]:

from PIL import ImageDraw

def json_to_mask(js_path):
    with open(js_path) as f:
        meta = json.load(f)
    h, w = meta['imageHeight'], meta['imageWidth']
    mask = Image.new('L', (w, h), 0)
    for shp in meta['shapes']:
        ImageDraw.Draw(mask).polygon(shp['points'], outline=1, fill=1)
    mask = mask.resize((IMG_SIZE, IMG_SIZE), Image.NEAREST)
    out_path = WORK_DIR / js_path.relative_to(DATA_DIR).with_suffix('.png').as_posix()
    out_path = out_path.replace('/annots/', '/masks/')
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    mask.save(out_path)

for split in ['train', 'valid']:
    for js in (DATA_DIR/split/'annots').glob('*.json'):
        json_to_mask(js)
print("✓ Masks generated in", WORK_DIR)

✓ Masks generated in D:\rico\archive\annotated_apple_leaf_disease


In [6]:
# Cell 3.1: Load & binarize a lesion mask
def load_mask(mask_path, size=(224, 224)):
    """Load mask as 2D binary array."""
    m = Image.open(mask_path).convert("L").resize(size, Image.NEAREST)
    arr = np.array(m)
    # Threshold at mid-point to get 0/1
    return (arr > 127).astype(np.uint8)


In [7]:

import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Collect image paths and labels
def collect_split(split: str):
    items = []
    for xml_fp in (DATA_DIR/split/'annots').glob('*.xml'):
        tree = ET.parse(xml_fp)
        root = tree.getroot()
        img_name = root.find('filename').text
        img_path = DATA_DIR/split/'images'/img_name
        for obj in root.findall('object'):
            lbl = obj.find('name').text
            items.append((str(img_path), lbl))
    return items

train_items = collect_split('train')
val_items   = collect_split('valid')

# Build class mapping
class_names = sorted({lbl for _, lbl in train_items})
class_to_idx = {c:i for i,c in enumerate(class_names)}

# Transforms
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

# Dataset class
class AppleLeafDataset(Dataset):
    def __init__(self, items, transform=None):
        self.items = items
        self.transform = transform

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        img_path, label = self.items[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        lbl = class_to_idx[label]
        return img, lbl

# DataLoaders _______.>
batch_size = 32
num_workers = min(4, os.cpu_count())

train_loader = DataLoader(AppleLeafDataset(train_items, transform), batch_size=batch_size,
                          shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader   = DataLoader(AppleLeafDataset(val_items, transform),   batch_size=batch_size,
                          shuffle=False,num_workers=num_workers,pin_memory=True)

print(f"✔ Classes: {class_names}")
print(f"Dataloaders ready: train batches = {len(train_loader)}, valid = {len(val_loader)}")

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [8]:
#  (Transfer Learning since no time ) & Training Loop
import torch.nn as nn
import torch.optim as optim
from torchvision import models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Base model
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

# Training loop (warm-up)
best_acc = 0.0
for epoch in range(5):
    model.train()
    running_loss = 0.0
    correct = total = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    train_loss = running_loss / total
    train_acc = correct / total

    # Validation
    model.eval()
    val_loss = val_corr = val_tot = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            l = criterion(out, labels)
            val_loss += l.item() * imgs.size(0)
            _, p = out.max(1)
            val_corr += (p == labels).sum().item()
            val_tot  += labels.size(0)
    val_loss /= val_tot
    val_acc = val_corr / val_tot

    print(f"Epoch {epoch+1}/5 — "
          f"train loss {train_loss:.4f}, acc {train_acc:.4f} | "
          f"val loss {val_loss:.4f}, acc {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_resnet50.pt')

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 141MB/s]
/usr/local/lib/python3.11/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


NameError: name 'train_loader' is not defined

In [9]:
#  tuning  Network
for param in model.parameters():
    param.requires_grad = True

optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(15):
    model.train()
    running_loss = correct = total = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer_ft.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_ft.step()
        running_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    train_loss = running_loss / total
    train_acc = correct / total

    # Validation same as above
    model.eval()
    val_loss = val_corr = val_tot = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            l = criterion(out, labels)
            val_loss += l.item() * imgs.size(0)
            _, p = out.max(1)
            val_corr += (p == labels).sum().item()
            val_tot  += labels.size(0)
    val_loss /= val_tot
    val_acc = val_corr / val_tot

    print(f"Epoch {epoch+1}/15 — "
          f"train loss {train_loss:.4f}, acc {train_acc:.4f} | "
          f"val loss {val_loss:.4f}, acc {val_acc:.4f}")

torch.save(model.state_dict(), 'leaf_classifier_final.pt')
print(" Model saved to leaf_classifier_final.pt")

NameError: name 'train_loader' is not defined

In [10]:
import torch.nn.functional as F

# Load  model
model.load_state_dict(torch.load('best_resnet50.pt'))
model.eval()

features = gradients = None
def forward_hook(module, inp, out):
    global features
    features = out.detach()

def backward_hook(module, grad_in, grad_out):
    global gradients
    gradients = grad_out[0].detach()

target_layer = model.layer4[-1].conv3
target_layer.register_forward_hook(forward_hook)
target_layer.register_backward_hook(backward_hook)

def make_gradcam_heatmap(img_tensor, thresh=0.5):
    model.zero_grad()
    preds = model(img_tensor)
    class_idx = preds.argmax(dim=1).item()
    preds[0, class_idx].backward()
    pooled_grads = gradients.mean(dim=[0,2,3])
    fmap = features[0]
    cam = (pooled_grads[:, None, None] * fmap).sum(dim=0).cpu().numpy()
    cam = np.maximum(cam, 0)
    cam = cam / (cam.max() + 1e-9)
    mask = (cam > thresh).astype(np.uint8) * 255
    return cam, mask

# Example visualization
sample_img, _ = next(iter(val_loader))
img_tensor = sample_img[0].unsqueeze(0).to(device)
raw = sample_img[0].permute(1,2,0).cpu().numpy()
heat, mask = make_gradcam_heatmap(img_tensor)

plt.figure(figsize=(8,3))
plt.subplot(1,3,1); plt.imshow(raw); plt.title('Leaf'); plt.axis('off')
plt.subplot(1,3,2); plt.imshow(mask, cmap='gray'); plt.title('Mask'); plt.axis('off')
plt.subplot(1,3,3); plt.imshow(heat, cmap='jet'); plt.title('Grad-CAM'); plt.axis('off')
plt.tight_layout(); plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'best_resnet50.pt'

In [11]:

import matplotlib.pyplot as plt

def validate_focus(img_path, mask_path, model, thresh=0.5):
    # 1) Load & preprocess image
    pil = Image.open(img_path).convert("RGB")
    inp = transform(pil).unsqueeze(0).to(device)
    raw = np.array(pil)

    # 2) Grad-CAM + binarize
    cam, bin_mask = make_gradcam_heatmap(inp, thresh=thresh)

    # 3) Ground-truth mask
    gt = load_mask(mask_path)

    # 4) Compute IoU
    iou = binary_iou(bin_mask, gt)
    print(f"IoU (Grad-CAM vs. GT): {iou:.4f}")

    # 5) Plot 4-panel figure
    fig, axs = plt.subplots(1, 4, figsize=(16,4))
    axs[0].imshow(raw);            axs[0].set_title("Original");           axs[0].axis("off")
    axs[1].imshow(gt, cmap="gray");axs[1].set_title("GT Lesion Mask");    axs[1].axis("off")
    axs[2].imshow(raw); axs[2].imshow(cam, cmap="jet", alpha=0.5);             axs[2].set_title("Grad-CAM Overlay");  axs[2].axis("off")
    axs[3].imshow(bin_mask, cmap="gray"); axs[3].set_title(f"Binarized (Th={thresh})\nIoU={iou:.2f}");    axs[3].axis("off")
    plt.tight_layout(); plt.show()

# Example on our dummy data:
validate_focus(sample_img_path, sample_mask_path, model)


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\rico\\archive\\annotated_apple_leaf_disease/dummy_leaf.png'

In [ ]:

from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, preds = outputs.max(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Create report & confusion matrix
report_dict = classification_report(all_labels, all_preds,
                                    target_names=class_names,
                                    output_dict=True)
cm = confusion_matrix(all_labels, all_preds)

# Convert to DataFrames
df_report = pd.DataFrame(report_dict).transpose()
df_cm     = pd.DataFrame(cm, index=class_names, columns=class_names)


df_report.to_csv('classification_report.csv', index=True)
df_cm.to_csv('confusion_matrix.csv', index=True)

# Display
print("=== Classification Report ===")
display(df_report.style.format("{:.4f}"))
print("\n=== Confusion Matrix ===")
display(df_cm)


In [ ]:

import zipfile

files_to_zip = [
    'classification_report.csv',
    'confusion_matrix.csv',
    'best_resnet50.pt',            # your saved best model
    'leaf_classifier_final.pt'     # final fine-tuned model , omg sathwik!
]

with zipfile.ZipFile('results_bundle.zip', 'w', zipfile.ZIP_DEFLATED) as zf:
    for fn in files_to_zip:
        if os.path.exists(fn):
            zf.write(fn)
        else:
            print(f"File not found, skipping: {fn}")

print("Packed files into results_bundle.zip")
